In [ ]:
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

Подключите необходимые библиотеки.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from matplotlib import colors
from scipy.stats import pearsonr, spearmanr, kendalltau
sns.set(rc={'figure.figsize':(25, 10)}); # you can change this if needed


Загрузите данные. Они находятся в директории input.

In [ ]:
df = pd.read_csv("../input/zomato.csv", encoding = "latin-1")
countries = pd.read_excel("../input/Country-Code.xlsx")
df = df.merge(countries, how="inner", on="Country Code")
df = df.dropna()
print(df.columns)

Целевой переменной будет рейтинг

В каких странах расположены рестораны? Назовите топ-5 стран. Выведите аналогичную статистику по городам.

In [ ]:
countries = df["Country"].value_counts().sort_values(ascending=False)
print("Топ стран по количеству ресторанов:\n", countries.head(5))
countries.head(5).plot(kind="bar", title="Top Countries by amount", color="blue")
plt.show()

In [ ]:
cities = df["City"].value_counts().sort_values(ascending=False)
print("\nТоп городов по количеству ресторанов:\n", cities.head(5))
cities.head(5).plot(kind="bar", title="Top Cities by amount", color="green")
plt.show()

Какие типы кухни подают в ресторанах? Как связан этот признак с рейтингом? Не забывайте о визуализации информации.

In [ ]:
cuisines = []
for i in df["Cuisines"].values:
    if str(i) != "nan":
        cuisines.extend(str(i).split(", "))
cuisines = np.unique(cuisines)
print(cuisines)


In [ ]:
cusrating = {}
crtable = df[["Aggregate rating", "Cuisines"]]
for i in cuisines:
    cusrating[i] = crtable[crtable["Cuisines"].str.contains(i)]["Aggregate rating"]
cusrating = pd.DataFrame(data=cusrating).mean().sort_values(ascending=False)
print(cusrating)

In [ ]:
cusrating.plot(kind="bar", fontsize = 8, color="green")
plt.show()

Что можно сказать о связи ценового диапазона и рейтинга ресторана? Не забывайте о визуализации информации.

In [ ]:
df.drop(df[(df["Votes"]<2)].index, inplace=True)
pricerating = df[["Price range", "Aggregate rating"]]
sns.barplot(y = "Price range", x = "Aggregate rating", color ="blue", data = pricerating)

In [ ]:
r = spearmanr(df["Price range"], df["Aggregate rating"])
print('Spearman correlation:', r[0], 'p-value:', r[1])

Есть ли различия в рейтинге ресторанов с возможностью бронирования столиков и без него? 

In [ ]:
r = spearmanr(df["Has Table booking"], df["Aggregate rating"])
print('Spearman correlation:', r[0], 'p-value:', r[1])

Визуализируйте признак **Rating color** (или **Rating text**) в комбинации с геолокацией (**Longitude**, **Latitude**). Какой вывод можно сделать?

In [ ]:
countries = gpd.read_file(
               gpd.datasets.get_path("naturalearth_lowres"))
countries.head()
countries.plot(color="lightgrey")
sns.scatterplot(y = 'Latitude', x = 'Longitude', hue = 'Rating text', data=df, s=50)

plt.show()

Можно сделать выводы: больше всего ресторанов в Индии и США; рестораны с рейтингом ниже "good" встречаются значительно реже, чем с рейтингом выше "average".

Какие ещё инсайты можно найти в имеющихся данных? Творческое задание.

In [ ]:
sns.pairplot(df[["Aggregate rating", 'Average Cost for two', 'Has Table booking', 'Has Online delivery', 'Price range', 'Votes']]);

In [ ]:
new_values = {'No':0, "Yes":1} # обычный словарь Python
df['delivery'] = df['Has Online delivery'].map(new_values)
df["booking"] = df['Has Table booking'].map(new_values)
df["switch_to_order"] = df["Switch to order menu"].map(new_values)
df[['Average Cost for two', "booking", 'delivery', "switch_to_order", 'Price range', 'Aggregate rating', 'Votes']].corr(method='spearman')


Полезными признаками для предсказания рейтинга могут быть "Votes", "Price range"